<a href="https://colab.research.google.com/github/chloevan/gcp_tutorial/blob/master/02_basics/Ch07_Split_Unnest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "Ch07 Arrays & Structs"
date: 2020-04-27T20:30:30+09:00
tags:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "빅쿼리 파이썬"
  - "빅쿼리 기본"
  - "SPLIT"
  - "UNNEST"
  - "BigQuery with Python"
categories:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "Python"
menu: 
  gcp:
    name: Ch06 SQL Aggregates
---

## 1. 구글 클라우드 설정

본격적인 빅쿼리 실습에 앞서서, Python과 연동하는 예제를 준비하였다. 빅쿼리 시작에 앞서서 선행적으로 클라우드 사용을 해야 한다. 

1. 만약 GCP 프로젝트가 없다면, 계정을 연동한다. Go to [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager)
2. 그리고, 비용결제를 위한 카드를 등록한다. [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) 
3. 마지막으로 BigQuery API를 사용해야 하기 때문에 빅쿼리 API 사용허가를 내준다.[Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery)

위 API를 이용하지 않으면 `Python` 또는 `R`과 연동해서 사용할 수는 없다. 자주 쓰는것이 아니라면 비용은 거의 발생하지 않으니 염려하지 않아도 된다. 비용관리에 대한 자세한 내용은 [BigQuery 권장사항: 비용 관리](https://cloud.google.com/bigquery/docs/best-practices-costs?hl=ko)에서 확인하기를 바란다.  


## 2. 사용자 계정 인증
구글 코랩을 사용해서 인증 절차를 밟도록 한다. 아래 소스코드는 변경시키지 않는다. 아래 절차대로 진행하면 된다. `Gmail` 인증 절차와 비슷하다.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## 3. 쿼리 기본
빅쿼리는 기본적으로 데이터 웨어하우스의 기능을 가지고 있다. Structured(데이터프레임) & Semi-Structured(JSON)과 같은 DB를 지원하고 있다. Create, Read, Update, Delete 기능을 지원한다. 

빅쿼리는 데이터 분석을 위한 기본적인 도구이기 때문에, 웹/앱을 활용한 마케팅 분석이 필요한 업종에서는 어쩌면 반드시 가져가야할 일종의 언어이기도 하다. 

기본적으로 SQL문법을 지원하며, BigQuery 나름의 문법을 또한 구성하고 있다. 이제 본격적으로 코드를 작성해보자. 

현재 작성하는 모든 코드는 Python-BigQuery로 연동하여 작성하려고 한다. 이제 간단하게 쿼리를 작성해보자. 

- [Ch05-Query_Essentials(1)](https://chloevan.github.io/gcp/bigquery/02_basics/ch05_query_essentials1/)에서는 `SELECT`와 `AS`에 대해 배우는 시간을 가졌다. 
- [Ch05-Query_Essentials(2)](https://chloevan.github.io/gcp/bigquery/02_basics/ch05_query_essentials2/)에서는 `WHERE`와 `EXCEPT`, 그리고 `REPLACE`에 대해 배우는 시간을 가졌다.
- [Ch05-Query_Essentials(3)](https://chloevan.github.io/gcp/bigquery/02_basics/ch05_query_essentials3/)에서는 `SubQuery`에 대해서 집중적으로 배우는 시간을 가졌다.  
- [Ch06 SQL Aggregates](https://chloevan.github.io/gcp/bigquery/02_basics/ch06_sql_aggregates/)에서는 집계함수를 사용해서 데이터를 조금 더 디테일하게 요약하는 것을 배웠다.

오늘 포스트에서는 `Split and Unnest`, `UNION`에 대해서 배우도록 한다. 오늘 포스트의 목적은 `테이블을 직접 만들어보자`이다. `(It's Free)`



### (1) Split and Unnest
데이터를 배열 형태로 조회할 수는 없을까? 빅쿼리에서는 이러한 문법을 보통 `Array`로 통칭해서 부른다. 

`ARRAY`와 `UNNEST`의 조합을 통해서, 사용자는 다양한 쿼리와 함수, 그리고 데이터 유형에 대해 사전에 먼저 실험 및 탐색할 수 있는 기회를 준다. 

우선, `Split`과 `UNNEST`을 활용해서 조회를 해보자. 

In [3]:
from google.cloud import bigquery
from tabulate import tabulate

project_id = 'bigquerytutorial-274406'
client = bigquery.Client(project=project_id)

temp = client.query('''
  SELECT 
    city, split(city, ' ') AS parts
  FROM (
    SELECT * from UNNEST([
      'Seattle WA', 'New York', 'Singapore'
    ]) AS city
  )
  ''').to_dataframe()

print(tabulate(temp, tablefmt="pipe", headers="keys"))

|    | city       | parts             |
|---:|:-----------|:------------------|
|  0 | Seattle WA | ['Seattle', 'WA'] |
|  1 | New York   | ['New', 'York']   |
|  2 | Singapore  | ['Singapore']     |


특히, `parts` Column을 보면, city `Seattle WA`가 ['Seattle', 'WA']로 반환되는 것을 볼 수 있다. 

위 코드가 좋은 이유는 빅쿼리에서는 이러한 형태로 데이터 셋을 만들어서 다양한 실험을 한 뒤, 실제 데이터에서 적용을 할 수 있는 장점이 있다. 이 때, 빅쿼리는 비용을 산정하지 않는다. 

즉, 빅쿼리가 JSON 형태의 데이터도 취급을 하기 때문에, 이렇게 `Arrays`를 잘 다룰줄 알면, 실제 데이터에 적용 시, 응용할 수 있는 장점을 제공한다. 

### (2) UNION
`Split`과 `Unnest`와 마찬가지로 임시적으로 데이터를 생성할 수 있다. 

소스 코드를 통해서 확인해보자. 

In [4]:
temp = client.query('''
  WITH example AS (
    SELECT 'Sat' As day, 1451 AS numrides,
    1018 AS oneways
    UNION ALL SELECT 'Sun', 2376, 936
    UNION ALL SELECT 'Mon', 1476, 736
  )

  SELECT * from example
  WHERE numrides < 2000
  ''').to_dataframe()

print(tabulate(temp, tablefmt='pipe', headers='keys'))


|    | day   |   numrides |   oneways |
|---:|:------|-----------:|----------:|
|  0 | Sat   |       1451 |      1018 |
|  1 | Mon   |       1476 |       736 |


마찬가지로 테이블이 생성된 것을 확인할 수 있다. 

매우 짧은 포스트이지만, 실무에서는 자주 사용될 수 있는 예제이다. 이제 본격적으로 `ARRAY`에 대해서 배우도록 하자. 

## 4. Reference
Lakshmanan, V., & Tigani, J. (2020). Google BigQuery: the definitive guide: data warehousing, analytics, and machine learning at scale. Beijing: OReilly.